### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [718]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Files to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [719]:
#Calculate the total number of schools
num_schools = len(school_data_complete["School ID"].unique())

#Calculate the total number of students
num_students = len(school_data_complete["Student ID"].unique())

#Calculate the total budget
tot_budget = school_data_complete["budget"].sum()

#Calculate the average math score
avg_math = school_data_complete["math_score"].mean()

#Calculate the average reading score
avg_read = school_data_complete["reading_score"].mean()

#Calculate the percentage of students with a passing math score (70 or greater)
math70 = len(school_data_complete.loc[(school_data_complete["math_score"] >= 70)])/num_students*100

#Calculate the percentage of students with a passing reading score (70 or greater)
read70 = len(school_data_complete.loc[(school_data_complete["reading_score"] >= 70)])/num_students*100

#Calculate the percentage of students who passed math and reading (% Overall Passing)
#This calculates only students who passed BOTH math and reading
rm70 = len(school_data_complete.loc[(school_data_complete["math_score"] >= 70) &
                                (school_data_complete["reading_score"] >= 70)])/num_students*100

#Create a dataframe to hold the above results
results_df = pd.DataFrame({"NumberOfSchools": [num_schools],
                          "NumberOfStudents": [num_students],
                          "AverageMathScore": [avg_math],
                          "AverageReadingScore": [avg_read],
                          "PctStudentsMathScore70Plus": [math70],
                          "PctStudentsReadingScore70Plus": [read70],
                          "PctStudentsMathAndReadingAbove70": [rm70]})

#Optional: give the displayed data cleaner formatting
results_df["NumberOfStudents"] = results_df["NumberOfStudents"].astype(int).map("{:,}".format)
results_df["AverageMathScore"] = results_df["AverageMathScore"].astype(float).map("{:,.2f}".format)
results_df["AverageReadingScore"] = results_df["AverageReadingScore"].astype(float).map("{:,.2f}".format)
results_df["PctStudentsMathScore70Plus"] = results_df["PctStudentsMathScore70Plus"].astype(float).map("{:,.2f}".format)
results_df["PctStudentsReadingScore70Plus"] = results_df["PctStudentsReadingScore70Plus"].astype(float).map("{:,.2f}".format)
results_df["PctStudentsMathAndReadingAbove70"] = results_df["PctStudentsMathAndReadingAbove70"].astype(float).map("{:,.2f}".format)
results_df

,NumberOfSchools,NumberOfStudents,AverageMathScore,AverageReadingScore,PctStudentsMathScore70Plus,PctStudentsReadingScore70Plus,PctStudentsMathAndReadingAbove70
0,15,"39,170",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [720]:
#Dataframe to hold School information
school_info_df = pd.DataFrame({"School ID": school_data_complete["School ID"],
                               "School Name": school_data_complete["school_name"],
                               "School Type": school_data_complete["type"],
                               "School Budget": school_data_complete["budget"]
                              })
school_info_df = school_info_df.drop_duplicates()
school_info_df.reset_index(inplace = True)

#Now add in students
school_group = school_data_complete.groupby("School ID")
tot_students = school_group["Student ID"].count()
school_info_df["Student Count"] = tot_students

#Find average math/reading scores
avg_math_score = school_group["math_score"].sum() / tot_students
avg_reading_score = school_group["reading_score"].sum() / tot_students
school_info_df["Average Math Score"] = avg_math_score
school_info_df["Average Reading Score"] = avg_reading_score

#% passing math
school_math_scores_df = school_data_complete[['School ID','math_score']]
school_math70_count = school_math_scores_df.loc[(school_math_scores_df["math_score"] >= 70)]
math_groups = school_math70_count.groupby('School ID')
math_pass = math_groups.count()
school_info_df["Num Pass Math"] =  math_groups.count()
school_info_df["Pct Pass Math"] = school_info_df["Num Pass Math"] / school_info_df["Student Count"] * 100

#% passing reading
school_reading_scores_df = school_data_complete[['School ID','reading_score']]
school_reading70_count = school_reading_scores_df.loc[(school_reading_scores_df["reading_score"] >= 70)]
reading_groups = school_reading70_count.groupby('School ID')
reading_pass = reading_groups.count()
school_info_df["Num Pass Reading"] =  reading_groups.count()
school_info_df["Pct Pass Reading"] = school_info_df["Num Pass Reading"] / school_info_df["Student Count"] * 100

#% passing both math and reading
school_both_scores_df = school_data_complete[['School ID','math_score','reading_score']]
school_both70_count = school_both_scores_df.loc[(school_both_scores_df["math_score"] >= 70) &
                                               (school_both_scores_df["reading_score"] >= 70)]
both_groups = school_both70_count.groupby('School ID')
both_pass = both_groups.count()
both70_pass = both_pass["math_score"]
school_info_df["Num Pass Both"] = both70_pass 
school_info_df["Pct Pass Both"] = school_info_df["Num Pass Both"] / school_info_df["Student Count"] * 100

#Create DataFrame to display the results
results_df = pd.DataFrame({"School Name": school_info_df["School Name"],
                          "School Type": school_info_df["School Type"],
                          "School Budget": school_info_df["School Budget"].astype(float).map("${:,.0f}".format),
                          "Student Count": school_info_df["Student Count"].astype(float).map("{:,.0f}".format),
                          "Per Student Budget": school_info_df["School Budget"] / school_info_df["Student Count"],
                           "Avg Math Score": school_info_df["Average Math Score"].astype(float).map("{:,.2f}".format),
                           "Avg Reading Score": school_info_df["Average Reading Score"].astype(float).map("{:,.2f}".format),
                           "Pct Pass Math": school_info_df["Pct Pass Math"].astype(float).map("{:,.2f}%".format),
                           "Pct Pass Reading": school_info_df["Pct Pass Reading"].astype(float).map("{:,.2f}%".format),
                           "Pct Pass Overall": school_info_df["Pct Pass Both"].astype(float).map("{:,.2f}%".format)
                          })
results_df["Per Student Budget"] = results_df["Per Student Budget"].astype(float).map("${:,.0f}".format)

results_df

,School Name,School Type,School Budget,Student Count,Per Student Budget,Avg Math Score,Avg Reading Score,Pct Pass Math,Pct Pass Reading,Pct Pass Overall
0,Huang High School,District,"$1,910,635","2,917",$655,76.63,81.18,65.68%,81.32%,53.51%
1,Figueroa High School,District,"$1,884,411","2,949",$639,76.71,81.16,65.99%,80.74%,53.20%
2,Shelton High School,Charter,"$1,056,600","1,761",$600,83.36,83.73,93.87%,95.85%,89.89%
3,Hernandez High School,District,"$3,022,020","4,635",$652,77.29,80.93,66.75%,80.86%,53.53%
4,Griffin High School,Charter,"$917,500","1,468",$625,83.35,83.82,93.39%,97.14%,90.60%
5,Wilson High School,Charter,"$1,319,574","2,283",$578,83.27,83.99,93.87%,96.54%,90.58%
6,Cabrera High School,Charter,"$1,081,356","1,858",$582,83.06,83.98,94.13%,97.04%,91.33%
7,Bailey High School,District,"$3,124,928","4,976",$628,77.05,81.03,66.68%,81.93%,54.64%
8,Holden High School,Charter,"$248,087",427,$581,83.80,83.81,92.51%,96.25%,89.23%
9,Pena High School,Charter,"$585,858",962,$609,83.84,84.04,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [721]:
sorted_df = results_df.sort_values("Pct Pass Overall", ascending=False)
sorted_df.head()

,School Name,School Type,School Budget,Student Count,Per Student Budget,Avg Math Score,Avg Reading Score,Pct Pass Math,Pct Pass Reading,Pct Pass Overall
6,Cabrera High School,Charter,"$1,081,356","1,858",$582,83.06,83.98,94.13%,97.04%,91.33%
14,Thomas High School,Charter,"$1,043,130","1,635",$638,83.42,83.85,93.27%,97.31%,90.95%
4,Griffin High School,Charter,"$917,500","1,468",$625,83.35,83.82,93.39%,97.14%,90.60%
5,Wilson High School,Charter,"$1,319,574","2,283",$578,83.27,83.99,93.87%,96.54%,90.58%
9,Pena High School,Charter,"$585,858",962,$609,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [722]:
sorted_df = results_df.sort_values("Pct Pass Overall")
sorted_df.head()

,School Name,School Type,School Budget,Student Count,Per Student Budget,Avg Math Score,Avg Reading Score,Pct Pass Math,Pct Pass Reading,Pct Pass Overall
11,Rodriguez High School,District,"$2,547,363","3,999",$637,76.84,80.74,66.37%,80.22%,52.99%
1,Figueroa High School,District,"$1,884,411","2,949",$639,76.71,81.16,65.99%,80.74%,53.20%
0,Huang High School,District,"$1,910,635","2,917",$655,76.63,81.18,65.68%,81.32%,53.51%
3,Hernandez High School,District,"$3,022,020","4,635",$652,77.29,80.93,66.75%,80.86%,53.53%
12,Johnson High School,District,"$3,094,650","4,761",$650,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [937]:
#DataFrame w/ School, Grade and Math scores
grade_math_score_df = pd.DataFrame({"School Name": school_data_complete["school_name"],
                   "Grade": school_data_complete["grade"],
                   "Math Score": school_data_complete["math_score"]})

group_math_score = grade_math_score_df.groupby(["School Name","Grade"])

math_average = round(group_math_score["Math Score"].mean(), 2)
math_average


School Name            Grade
Bailey High School     10th     77.00
                       11th     77.52
                       12th     76.49
                       9th      77.08
Cabrera High School    10th     83.15
                       11th     82.77
                       12th     83.28
                       9th      83.09
Figueroa High School   10th     76.54
                       11th     76.88
                       12th     77.15
                       9th      76.40
Ford High School       10th     77.67
                       11th     76.92
                       12th     76.18
                       9th      77.36
Griffin High School    10th     84.23
                       11th     83.84
                       12th     83.36
                       9th      82.04
Hernandez High School  10th     77.34
                       11th     77.14
                       12th     77.19
                       9th      77.44
Holden High School     10th     83.43
                     

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [772]:
#DataFrame w/ School, Grade and Reading scores
grade_reading_score_df = pd.DataFrame({"School Name": school_data_complete["school_name"],
                   "Grade": school_data_complete["grade"],
                   "Reading Score": school_data_complete["reading_score"]})

group_reading_score = grade_reading_score_df.groupby(["School Name","Grade"])

reading_average = round(group_reading_score["Reading Score"].mean(), 2)

reading_df = pd.DataFrame({"Reading Average": reading_average})
reading_df

Reading Average
School Name           Grade                 
Bailey High School    10th             80.91
                      11th             80.95
                      12th             80.91
                      9th              81.30
Cabrera High School   10th             84.25
                      11th             83.79
                      12th             84.29
                      9th              83.68
Figueroa High School  10th             81.41
                      11th             80.64
                      12th             81.38
                      9th              81.20
Ford High School      10th             81.26
                      11th             80.40
                      12th             80.66
                      9th              80.63
Griffin High School   10th             83.71
                      11th             84.29
                      12th             84.01
                      9th              83.37
Hernandez High School 10th             80.66
                      11th             81.40
                      12th             80.86
                      9th              80.87
Holden High School    10th             83.32
                      11th             83.82
                      12th             84.70
                      9th              83.68
Huang High School     10th             81.51
                      11th             81.42
                      12th             80.31
                      9th              81.29
Johnson High School   10th             80.77
                      11th             80.62
                      12th             81.23
                      9th              81.26
Pena High School      10th             83.61
                      11th             84.34
                      12th             84.59
                      9th              83.81
Rodriguez High School 10th             80.63
                      11th             80.86
                      12th             80.38
                      9th              80.99
Shelton High School   10th             83.44
                      11th             84.37
                      12th             82.78
                      9th              84.12
Thomas High School    10th             84.25
                      11th             83.59
                      12th             83.83
                      9th              83.73
Wilson High School    10th             84.02
                      11th             83.76
                      12th             84.32
                      9th              83.94
Wright High School    10th             83.81
                      11th             84.16
                      12th             84.07
                      9th              83.83

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [790]:
school_info_df["Per Student Budget"] = school_info_df["School Budget"] / school_info_df["Student Count"]

spending_df = pd.DataFrame({"School Name": school_info_df["School Name"],
                          "Per Student Budget": school_info_df["Per Student Budget"],
                           "Avg Math Score": school_info_df["Average Math Score"].astype(float).map("{:,.2f}".format),
                           "Avg Reading Score": school_info_df["Average Reading Score"].astype(float).map("{:,.2f}".format),
                           "Pct Pass Math": school_info_df["Pct Pass Math"].astype(float).map("{:,.2f}%".format),
                           "Pct Pass Reading": school_info_df["Pct Pass Reading"].astype(float).map("{:,.2f}%".format),
                           "Pct Pass Overall": school_info_df["Pct Pass Both"].astype(float).map("{:,.2f}%".format)
                          })

bins = [0,599,624,649,700]
group_names = ["0 to 599", "600 to 624","625 to 649","650 to 700"]

spending_df["Spend Range"] = pd.cut(spending_df["Per Student Budget"], bins, labels=group_names, include_lowest=True)
sort_spending_df = spending_df.sort_values(["Spend Range"], ascending=True)
organized_df = sort_spending_df[["Spend Range", 
                                 "School Name", 
                                 "Avg Math Score",
                                 "Avg Reading Score",
                                 "Pct Pass Math",
                                 "Pct Pass Reading",
                                 "Pct Pass Overall"]]
organized_df

,Spend Range,School Name,Avg Math Score,Avg Reading Score,Pct Pass Math,Pct Pass Reading,Pct Pass Overall
5,0 to 599,Wilson High School,83.27,83.99,93.87%,96.54%,90.58%
6,0 to 599,Cabrera High School,83.06,83.98,94.13%,97.04%,91.33%
8,0 to 599,Holden High School,83.80,83.81,92.51%,96.25%,89.23%
10,0 to 599,Wright High School,83.68,83.95,93.33%,96.61%,90.33%
2,600 to 624,Shelton High School,83.36,83.73,93.87%,95.85%,89.89%
9,600 to 624,Pena High School,83.84,84.04,94.59%,95.95%,90.54%
1,625 to 649,Figueroa High School,76.71,81.16,65.99%,80.74%,53.20%
4,625 to 649,Griffin High School,83.35,83.82,93.39%,97.14%,90.60%
7,625 to 649,Bailey High School,77.05,81.03,66.68%,81.93%,54.64%
11,625 to 649,Rodriguez High School,76.84,80.74,66.37%,80.22%,52.99%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [791]:
school_sz_df = pd.DataFrame({"School Name": school_info_df["School Name"],
                          "Student Count": school_info_df["Student Count"],
                           "Avg Math Score": school_info_df["Average Math Score"].astype(float).map("{:,.2f}".format),
                           "Avg Reading Score": school_info_df["Average Reading Score"].astype(float).map("{:,.2f}".format),
                           "Pct Pass Math": school_info_df["Pct Pass Math"].astype(float).map("{:,.2f}%".format),
                           "Pct Pass Reading": school_info_df["Pct Pass Reading"].astype(float).map("{:,.2f}%".format),
                           "Pct Pass Overall": school_info_df["Pct Pass Both"].astype(float).map("{:,.2f}%".format)
                          })

bins = [0,999,1999,2999,5000]
group_names = ["0 to 999", "1000 to 1999","2000 to 2999","3000 to 5000"]

school_sz_df["Student Count"] = pd.cut(school_sz_df["Student Count"], bins, labels=group_names, include_lowest=True)
sort_sz_df = school_sz_df.sort_values(["Student Count"], ascending=True)
organized_df = sort_sz_df[["Student Count", 
                                 "School Name", 
                                 "Avg Math Score",
                                 "Avg Reading Score",
                                 "Pct Pass Math",
                                 "Pct Pass Reading",
                                 "Pct Pass Overall"]]
organized_df

,Student Count,School Name,Avg Math Score,Avg Reading Score,Pct Pass Math,Pct Pass Reading,Pct Pass Overall
8,0 to 999,Holden High School,83.80,83.81,92.51%,96.25%,89.23%
9,0 to 999,Pena High School,83.84,84.04,94.59%,95.95%,90.54%
2,1000 to 1999,Shelton High School,83.36,83.73,93.87%,95.85%,89.89%
4,1000 to 1999,Griffin High School,83.35,83.82,93.39%,97.14%,90.60%
6,1000 to 1999,Cabrera High School,83.06,83.98,94.13%,97.04%,91.33%
10,1000 to 1999,Wright High School,83.68,83.95,93.33%,96.61%,90.33%
14,1000 to 1999,Thomas High School,83.42,83.85,93.27%,97.31%,90.95%
0,2000 to 2999,Huang High School,76.63,81.18,65.68%,81.32%,53.51%
1,2000 to 2999,Figueroa High School,76.71,81.16,65.99%,80.74%,53.20%
5,2000 to 2999,Wilson High School,83.27,83.99,93.87%,96.54%,90.58%


## Scores by School Type

* Perform the same operations as above, based on school type

In [921]:
#school_info_df
school_type_df = pd.DataFrame({"School Type": school_data_complete["type"],
                           "Math Score": school_data_complete["math_score"],
                           "Reading Score": school_data_complete["reading_score"]
                          })

#Group the data by school type
group_school_type = school_type_df.groupby(["School Type"])
school_type_reading_avg = group_school_type["Reading Score"].mean()

#Number of students per school type
type_count = group_school_type.count()

school_type_list = school_data_complete["type"].unique()

for type in school_type_list:
    #generate a count of all reading scores >= 70
    type_reading70_count = school_type_df.loc[(school_type_df["Reading Score"] >= 70) &
                                             (school_type_df["School Type"] == type)]
    reading70_count = len(type_reading70_count)
    #print(type + str(reading70_count))
    school_type_summary_df = pd.DataFrame({"School Type": [type]})




#Number of students >= 70
#type_reading70_count = school_type_df.loc[(school_type_df["reading_score"] >= 70)]
#reading_groups = school_reading70_count.groupby('School ID')
#reading_pass = reading_groups.count()





#Setup school_type_summary_df for final report
#Now display the data in a report grouped by School Type
school_type_summary_df["Reading Average"] =  ({[school_type_reading_avg]})
                                      #"Math Average": school_type_math_avg.astype(float).map("{:,.2f}".format)})

#print(type + "   " + str(reading70_count))
school_type_summary_df
#Output:
#School Type	Avg Math Score	Avg Reading Score	Pct Pass Math	Pct Pass Reading	Pct Pass Overall

TypeError: unhashable type: 'list'